In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import cv2
import math
import time

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import sys
sys.path.append('..')
from detector.input_pipeline import KeypointPipeline

In [ ]:
cmap = plt.cm.get_cmap('autumn')
new_cmap = cmap(np.arange(cmap.N))
new_cmap[:, -1] = np.sqrt(np.linspace(0, 1, cmap.N))  # set alpha
cmap = ListedColormap(new_cmap)  # create new colormap

# Build a graph

In [ ]:
tf.reset_default_graph()

files = [
    '/home/dan/datasets/COCO/multiposenet/train/shard-0001.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0002.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0003.tfrecords',
]
params = {
    'batch_size': 16, 'image_height': 512, 'image_width': 512, 'min_dimension': 512
}

with tf.device('/cpu:0'):
    pipeline = KeypointPipeline(files, is_training=True, params=params)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()
    features.update(labels)

# Show an image

In [ ]:
ORDER = {
    0: 'nose',
    1: 'left eye', 2: 'right eye',
    3: 'left ear', 4: 'right ear',
    5: 'left shoulder', 6: 'right shoulder',
    7: 'left elbow', 8: 'right elbow',
    9: 'left wrist', 10: 'right wrist',
    11: 'left hip', 12: 'right hip',
    13: 'left knee', 14: 'right knee',
    15: 'left ankle', 16: 'right ankle'
}


def plot_maps(image, heatmaps, segmentation_mask, loss_mask):
    """
    Arguments:
        image: a float numpy array with shape [h, w, 3].
        heatmaps: a float numpy array with shape [h / 4, w / 4, 17].
        segmentation_mask: a float numpy array with shape [h / 4, w / 4].
        loss_mask: a float numpy array with shape [h / 4, w / 4].
    """

    h, w, _ = image.shape
    h, w = (h // 2), (w // 2)
    background = Image.new('RGBA', (w, h * 19), (255, 255, 255, 255))
    draw = ImageDraw.Draw(background, 'RGBA')
    
    image = (255*image).astype('uint8')
    image = Image.fromarray(image)
    image = image.resize((w, h), Image.LANCZOS)
    image.putalpha(255)

    heatmaps = (255*cmap(heatmaps)).astype('uint8')
    # it has shape [h, w, 17, 4]
    
    heats = []
    for j, name in ORDER.items():

        heat = Image.fromarray(heatmaps[:, :, j])
        heat = heat.resize((w, h), Image.LANCZOS)
        heat = Image.alpha_composite(image, heat)
        background.paste(heat, (0, j*h))
        draw.text((0, j*h), name, fill='red')
        
    def draw_mask(mask):
        mask = np.clip(mask, 0.0, 1.0)
        mask = (255*mask).astype('uint8')
        mask = Image.fromarray(mask)
        mask = mask.resize((w, h), Image.LANCZOS).convert('RGB')
        mask.putalpha(mask.convert('L'))
        mask = Image.alpha_composite(image, mask)
        return mask
    
    mask = draw_mask(segmentation_mask)
    background.paste(mask, (0, 17*h))
    draw.text((0, 17*h), 'segmentation mask', fill='red')
    
    mask = draw_mask(loss_mask)
    background.paste(mask, (0, 18*h))
    draw.text((0, 18*h), 'loss mask', fill='red')

    return background

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run(features)

In [ ]:
i = 0
image = output['images'][i]
heatmaps = output['heatmaps'][i]
segmentation_mask = output['segmentation_masks'][i]
loss_mask = output['loss_masks'][i]

plot_maps(image, heatmaps, segmentation_mask, loss_mask)

# Measure speed

In [ ]:
times = []
with tf.Session() as sess:
    sess.run(init)
    for _ in range(105):
        start = time.perf_counter()
        output = sess.run(features)
        times.append(time.perf_counter() - start)

times = np.array(times[5:])
print(times.mean(), times.std())